In [ ]:
from google.colab import files
myfile = files.upload()

In [ ]:
import io
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
df = pd.read_csv(io.BytesIO(myfile['vgames2.csv']))

In [ ]:
##타입확인 Year빼고 전부 object 타입.
df.dtypes

In [ ]:
##결측치 확인.

df.isnull().sum()

In [ ]:
##결측치확인
df['Year'].isnull()

In [ ]:
##행이 너무 많아 결측치 행만 확인
df['Year'][df['Year'].isnull()]

## df['Genre'][df['Genre'].isnull()]
## df['Publisher'][df['Publisher'].isnull()]

In [ ]:
##0번째 컬럼 제거
df.drop(df.columns[0], axis = 1 , inplace = True)
##NNaN값 행들 제거 데이터 크기에 비해 결측값이 적어 그냥 제거
df = df.dropna()

In [ ]:
df.head(20)
##확인해보니 Sales의 단위들이 맞지않음
##year가 플롯 타입 이여서 .0이 보임

In [ ]:
#k 가 들어가 있는행을 살려 *0.001 로해 m단위로 변경하려 했지만 비교결과 전체 데이터에 비해 상대적으로 행이 많지 않아 그냥 드랍시켜도 오차는 미미했음
# 지역별 Sales 드랍
drop_values = ['M','K']
df = df[~df['NA_Sales'].str.contains('|'.join(drop_values))]
df = df[~df['EU_Sales'].str.contains('|'.join(drop_values))]
df = df[~df['JP_Sales'].str.contains('|'.join(drop_values))]
df = df[~df['Other_Sales'].str.contains('|'.join(drop_values))]
df

In [ ]:
##타입변경
df['NA_Sales'] = pd.to_numeric(df['NA_Sales'])
df['EU_Sales'] = pd.to_numeric(df['EU_Sales'])
df['JP_Sales'] = pd.to_numeric(df['JP_Sales'])
df['Other_Sales'] = pd.to_numeric(df['Other_Sales'])

df.info()

In [ ]:
#년도별 게임 트렌드를 위한 Year 전처리
def year(x):
  if (x >= 0) & (x < 20):
    return int(2000 + x)
  elif (x > 30) & (x < 1000):
    return int(1900 + x)
  else:
    return int(x)

df['Year'] = df['Year'].apply(year)
df

In [ ]:
##바뀐타입확인
df.dtypes

In [ ]:
# Sales 모두 합친 컬럼생성 전체 출고량
df['Total_Sales'] = df['NA_Sales']+df['EU_Sales']+df['JP_Sales']+df['Other_Sales']

In [ ]:
#지역별 장르

df_genre_NA = df.groupby(['Genre'])['NA_Sales'].sum()
df_genre_EU = df.groupby(['Genre'])['EU_Sales'].sum()
df_genre_JP = df.groupby(['Genre'])['JP_Sales'].sum()
df_genre_Other = df.groupby(['Genre'])['Other_Sales'].sum()
df_genre_Total = df.groupby(by=['Genre'])['Total_Sales'].sum()
df_genre = pd.DataFrame()
df_genre = df_genre.append(df_genre_NA)
df_genre = df_genre.append(df_genre_EU)
df_genre = df_genre.append(df_genre_JP)
df_genre = df_genre.append(df_genre_Other)
df_genre = df_genre.append(df_genre_Total)
df_genre = df_genre.T
df_genre

In [ ]:
###지역별 매출량 시각화
sns.set(style='darkgrid')
plt.figure(figsize=(15, 10))
plt.title("local_sales", fontsize=30)
plt.xlabel("Genrne", fontsize=20)
plt.ylabel("sales", fontsize=20)

sns.lineplot(data=df_genre, markers=True);

In [ ]:
## 1위는 액션, 2위는 스포츠, 3위는 슈팅이다.
## 특이사항으로는 일본 지역만 롤플레잉이 1위이다.

In [ ]:
#카이제곱검정 독립변수(장르) 종속변수 (국가별 판매량)
##귀무가설 : 비율 차이가 없다. (=지역마다 선호하는 장르가 같다.)
from scipy.stats import chi2_contingency
chi2_val, p, dof, expected= chi2_contingency(df_genre, correction=False)
if(p<0.05) : 
    print('p value:'+'귀무가설 기각. 지역마다 선호하는 장르가 다름.')
else :
    print('p value:'+'귀무가설 기각 못함. 지역마다 선호하는 장르 같음.')

In [ ]:
##연도별 장르합
df_year_sale = df.groupby(by=['Year', 'Genre'])['Total_Sales'].sum().reset_index()
year_sale = df_year_sale.groupby(by=['Year'])['Total_Sales'].transform(max) == df_year_sale['Total_Sales']
year_sale_max = df_year_sale[year_sale].reset_index(drop=True)

genre = year_sale_max['Genre']

plt.figure(figsize=(30, 18))
g = sns.barplot(x='Year', y='Total_Sales', data=year_sale_max)
index = 0
for value in year_sale_max['Total_Sales']:
    g.text(index, value + 1, str(genre[index] + '-' +str(round(value, 2))), color='#000', size=20, rotation= 90, ha="center")
    index += 1

plt.title("Yearly game trends", fontsize=30)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Total sales volume", fontsize=20)

plt.xticks(rotation=90)
plt.show()

In [ ]:
## 2000년 까지는 대체적으로 매년 다양하게 수요가 있었다.
## 2001년 부터 액션게임의 수요가 매우높다
## 2006년과 2009년에는 스포츠장르의 수요가 액션 장르를 이겼다

In [ ]:
## 최근 트렌드를 분석하기위해 2010년 이후의 데이터 100개로 출고량이 많았던 플랫폼과 장르를 조사해보았다.
##sales_100
sales_100 = df[df.Year > 2010].sort_values(by='Total_Sales' ,ascending=False)
sales_100 = sales_100.head(100).reset_index(drop=True)
sales_100

In [ ]:
# Platform 100 그래프
platform_100 = sales_100.groupby(['Platform'])['Total_Sales'].sum().reset_index()
platform1 = platform_100['Platform'].values
plt.figure(figsize=(15, 10))
platform_sales_100 = sns.barplot(x ='Total_Sales', y='Platform', data=platform_100)
cnt = 0
for value in platform_100['Total_Sales']:
    platform_sales_100.text(x=value + 5, y=cnt, s=str(round(value,2)),
            color='black', size=10)
    cnt+=1
plt.title("platform_sales 100", fontsize=30)
plt.xlabel('sales', fontsize=20)
plt.ylabel('platform', fontsize=20)
plt.show()

In [ ]:
# 최근 10년간 출고량이 많았던 플랫폼은 PS3,X360,ps4이다.

In [ ]:
# Genre100 그래프
genre_100 = sales_100.groupby(['Genre'])['Total_Sales'].sum().reset_index()
genre = genre_100['Genre'].values
plt.figure(figsize=(10, 10))
genre_sales_100 = sns.barplot(x ='Total_Sales', y='Genre', data=genre_100)
cnt = 0
for value in genre_100['Total_Sales']:
    genre_sales_100.text(x=value + 5, y=cnt, s=str(round(value,2)),
            color='black', size=10)
    cnt+=1
plt.title('genre_sales 100', fontsize=30)
plt.xlabel('sales', fontsize=30)
plt.ylabel('genre', fontsize=30)
plt.show()

In [ ]:
# 최근 10년간 출고량이 많았던 장르는 Shooter,Action,Role=Playing 이다.

In [ ]:
##문득 100개를 하던 도중 1000개로 하면 결과가 달라질것 같은 생각이 들어서 1000개로 해보았다.
sales_1000 = df[df.Year > 2010].sort_values(by='Total_Sales' ,ascending=False)
sales_1000 = sales_1000.head(1000).reset_index(drop=True)
sales_1000

In [ ]:
# top1000 Platform 그래프
platform_1000 = sales_1000.groupby(['Platform'])['Total_Sales'].sum().reset_index()
platform2 = platform_1000['Platform'].values
plt.figure(figsize=(10, 10))
platform_sales_1000 = sns.barplot(x ='Total_Sales', y='Platform', data=platform_1000)
cnt = 0
for value in platform_1000['Total_Sales']:
    platform_sales_1000.text(x=value + 5, y=cnt, s=str(round(value,2)),
            color='black', size=10)
    cnt+=1
plt.title('platform_sales 1000', fontsize=30)
plt.xlabel('sales', fontsize=30)
plt.ylabel('platform', fontsize=30)
plt.show()

In [ ]:
##플랫폼은 변화가없다.

In [ ]:
# Genre1000 그래프
genre_1000 = sales_1000.groupby(['Genre'])['Total_Sales'].sum().reset_index()
genre2 = genre_1000['Genre'].values
plt.figure(figsize=(10, 10))
genre_sales_1000 = sns.barplot(x ='Total_Sales', y='Genre', data=genre_1000)
cnt = 0
for value in genre_1000['Total_Sales']:
    genre_sales_1000.text(x=value + 5, y=cnt, s=str(round(value,2)),
            color='black', size=10)
    cnt+=1
plt.title('genre_sales 1000', fontsize=30)
plt.xlabel('sales', fontsize=30)
plt.ylabel('genre', fontsize=30)
plt.show()

In [ ]:
## 장르는 변화가 있었다. 100개로 하였을 때에는 1. 슈팅 2. 액션 3.롤플레잉 이였으나
## 1000개로 하였을 때에는 1.액션 2. 슈팅 3.스포츠 였다. 그리하여 데이터가 더 많으면 어떨까 싶어 최근 2010년 이후의 데이터들이 몇개가 있을까 보았더니

In [ ]:
sales_2010 = df[df.Year > 2010].sort_values(by='Total_Sales' ,ascending=False)
sales_2010 = sales_2010.reset_index(drop=True)
sales_2010

In [ ]:
##처음에 drop한 데이터를 빼고 3685개가 있었다 그리하여 3685개를 다 넣어봤다.

In [ ]:
##sales_all
sales_all = df[df.Year > 2010].sort_values(by='Total_Sales' ,ascending=False)
sales_all = sales_all.reset_index(drop=True)

In [ ]:
# Platform all 그래프
platform_all = sales_all.groupby(['Platform'])['Total_Sales'].sum().reset_index()
platform3 = platform_all['Platform'].values
plt.figure(figsize=(15, 10))
platform_sales_all = sns.barplot(x ='Total_Sales', y='Platform', data=platform_all)
cnt = 0
for value in platform_all['Total_Sales']:
    platform_sales_all.text(x=value + 5, y=cnt, s=str(round(value,2)),
            color='black', size=10)
    cnt+=1
plt.title('platform_sales all', fontsize=30)
plt.xlabel('sales', fontsize=30)
plt.ylabel('platform', fontsize=30)
plt.show()

In [ ]:
##플랫폼은 1000과 비교했을때 별차이 없지만 3DS가 3위의 PS4를 바짝 추격하였다.

In [ ]:
# Genre_all 그래프
genre_all = sales_all.groupby(['Genre'])['Total_Sales'].sum().reset_index()
genre3 = genre_all['Genre'].values
plt.figure(figsize=(15, 10))
genre_sales_all = sns.barplot(x ='Total_Sales', y='Genre', data=genre_all)
cnt = 0
for value in genre_all['Total_Sales']:
    genre_sales_all.text(x=value + 5, y=cnt, s=str(round(value,2)),
            color='black', size=10)
    cnt+=1
plt.title('genre_sales all', fontsize=30)
plt.xlabel('sales', fontsize=30)
plt.ylabel('genre', fontsize=30)
plt.show()

In [ ]:
##1위가 액션으로 압도적으로 높았고 2위가 슈팅으로 높았다 3위도 스포츠로 장르1000과 별차이없지만 롤플레잉이 3위 스포츠와 비슷할정도로 많이 늘어났다.

In [ ]:
## 다음 게임은 어떤 특성을 가져야 할까?
## 플랫폼:  PS3, PS4, X360

## 장르:  액션, 슈팅, 스포츠

## 지역:  남미, 유럽

##으로 결론이 났습니다~